In [1]:
#Script para gerar a tabela Categoria

import pandas as pd
from datetime import date

categorias= ["Mercearia","Limpeza","Higiene Pessoal","Bebidas","Bebidas Alcoolicas","Laticinios","Carnes","Congelados",
             "Enlatados","Hortifruti","Padaria","Utensilios"]

cat_descr = ["Produtos não perecíveis, que não precisam de refrigeração",
             "Produtos relacionados à todo tipo de limpeza",
             "Produtos que servem para limpar e cuidar do corpo",
             "Bebidas nao alcoolicas: águas, refrigerantes, sucos",
             "Bebidas alcoolicas: vinhos, cervejas, bebidas destiladas",
             "Produtos alimentícios derivados do leite",
             "Acougue",
             "Produtos ultraprocessados, congelados e prontos para o consumo",
             "Produtos em conserva e enlatados",
             "Produtos hortifrutiganjeiro, altamente perecíveis",
             "Tipos de pães, torradas, bolos, petit-fours, salgados, doces e confeitos.",
             "Itens de utilidades domésticas, como utensílios para cozinha, papelaria, artigos para festa, brinquedos"
             
]

#id_categoria = [indice + 1 for indice in range(len(categorias))]


df_categoria = pd.DataFrame({
                              #"IDCATEGORIA":  id_categoria,
                              "NOME":         categorias,
                              "DESCRICAO":    cat_descr,  
                              "DATA_CRIACAO": date.today().strftime('%d/%m/%Y'),
                              "DATA_ATLZ":    "",
                              "VERSAO":       1

})

df_categoria.head(15)

caminho = r"C:\\Users\\marce\\OneDrive\\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\BANCO DE DADOS\\DADOS\\CATEGORIA\\"
arquivo = r"CATEGORIA.xlsx"
caminho_completo = caminho+arquivo

df_categoria.to_excel(caminho_completo,index=False)


In [2]:
#Script para enviar tabela xlsx para SQL da CATEGORIA

import pandas as pd
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text

conexaoDB = ('DRIVER={ODBC Driver 17 for SQL Server};SERVER=DESKTOP-HV85ADA\\PROJETO_MERCADO;DATABASE=MERCADO_OLTP;Trusted_Connection=yes;')
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={conexaoDB}')

caminho = r"C:\\Users\\marce\\OneDrive\\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\BANCO DE DADOS\\DADOS\\CATEGORIA\\"
arquivo = r"CATEGORIA.xlsx"
caminho_completo = caminho+arquivo

dados = pd.read_excel(caminho_completo)

tipo_dados = {

    #'IDCATEGORIA':  INTEGER,
    'NOME':         VARCHAR(50),
    'DESCRICAO':    VARCHAR(150),
    'DATA_CRIACAO': DATE,
    'DATA_ATLZ':    DATE,
    'VERSAO':       INTEGER

}

with engine.connect() as conn:
        conn.execute(text(f"TRUNCATE TABLE CATEGORIA"))  #limpa os dados existentes, pode ser removido do codigo
        conn.commit()

dados.to_sql(name='CATEGORIA',con=engine,if_exists='append',index=False,dtype=tipo_dados)

engine.dispose()

In [8]:
#Script para gerar a tabela Categoria e enviar diretamente para o SQL server

import pandas as pd
from datetime import date
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, VARCHAR, DATE,INTEGER, text
import pyodbc

categorias= ["Mercearia","Limpeza","Higiene Pessoal","Bebidas","Bebidas Alcoolicas","Laticinios","Carnes","Congelados",
             "Enlatados","Hortifruti","Padaria","Utensilios"]

cat_descr = ["Produtos não perecíveis, que não precisam de refrigeração",
             "Produtos relacionados à todo tipo de limpeza",
             "Produtos que servem para limpar e cuidar do corpo",
             "Bebidas nao alcoolicas: águas, refrigerantes, sucos",
             "Bebidas alcoolicas: vinhos, cervejas, bebidas destiladas",
             "Produtos alimentícios derivados do leite",
             "Acougue",
             "Produtos ultraprocessados, congelados e prontos para o consumo",
             "Produtos em conserva e enlatados",
             "Produtos hortifrutiganjeiro, altamente perecíveis",
             "Tipos de pães, torradas, bolos, petit-fours, salgados, doces e confeitos.",
             "Itens de utilidades domésticas, como utensílios para cozinha, papelaria, artigos para festa, brinquedos"
             
]

df_categoria = pd.DataFrame({
                              "NOME":         categorias,
                              "DESCRICAO":    cat_descr,  
                              "DATA_CRIACAO": date.today().strftime('%d/%m/%Y'),
                              "DATA_ATLZ":    "",
                              "VERSAO":       1

})

df_categoria.head(15)

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"
#user = ""
#senha = ""

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
#conn_str = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}"
engine = create_engine(conexaostring)

nome_tabela = "CATEGORIA"

#Usar paar nomear as colunas do dataframe com os mesmos nomes das colunas do banco de dados
#df_CFOP.rename(columns={'Coluna1': 'COD_CFOP', 'Coluna2': 'DESCRICAO'}, inplace=True)

tipo_dados = {

    "NOME":         VARCHAR(20),
    "DESCRICAO":    VARCHAR(20),
    "DATA_CRIACAO": DATE,
    "DATA_ATLZ":    DATE,
    "VERSAO":       INTEGER,
    
}

try:
    with engine.connect() as conn:
        conn.execute(text(f"TRUNCATE TABLE {nome_tabela}"))  #limpa os dados existentes, pode ser removido do codigo
        conn.commit()

    df_categoria.to_sql(
                            name=nome_tabela,
                            con=engine,
                            if_exists="append", #se usar o replace ele faz um drop table da tabela no banco de dados e cria uma com a estrutura do dataframe
                            index=False,
                            dtype=tipo_dados,
)
    print("Dados inseridos com sucesso")
except Exception as e:
    print(f"Erro ao inserir dados: {e}")